# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

external_data = "output_data/cities.csv"

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
cities = pd.read_csv(external_data)
cities_df = pd.DataFrame(cities)
cities_df

,Unnamed: 0,City,Clouds,Country,Date,Latitude,Longitude,High Temperature,Wind Speed,Humidity
0,0,sao francisco,10,BR,1618767612,-15.9486,-44.8644,33.49,2.43,25
1,1,port alfred,0,ZA,1618767613,-33.5906,26.8910,23.13,2.41,55
2,2,grindavik,75,IS,1618767614,63.8424,-22.4338,3.00,14.40,87
3,3,carnarvon,0,AU,1618767615,-24.8667,113.6333,24.27,8.47,71
4,4,romilly-sur-seine,0,FR,1618767617,48.5108,3.7263,13.00,2.57,50
...,...,...,...,...,...,...,...,...,...,...
548,548,lakes entrance,4,AU,1618768304,-37.8811,147.9810,6.67,2.65,94
549,549,asyut,0,EG,1618768305,27.1810,31.1837,36.00,6.69,21
550,550,kushiro,64,JP,1618768306,42.9750,144.3747,4.79,2.44,95
551,551,fukue,80,JP,1618768308,32.6881,128.8419,13.48,6.59,51


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
#Configuring Google Maps for overlaying layers
gmaps.configure(api_key = g_key)

coords = cities[['Latitude','Longitude']].astype(float)
humidity = cities['Humidity'].astype(float)

In [27]:
#Basis for heat layer mapping
hmap = gmaps.figure()

#Creating a layer
heat_layer = gmaps.heatmap_layer(coords, weights = humidity, dissipating = False,
                              max_intensity = 75, point_radius = 2)

#Adding the created layer heat_layer
hmap.add_layer(heat_layer)

#Display hmap
hmap

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.

* Drop any rows will null values.

In [55]:
#setting weather parameters of my choice.  Next iteration will allow for user input
weather_choice = cities.loc[(cities['Humidity'] < 70) & 
                           (cities['Wind Speed'] > 10) &
                           (cities['High Temperature'] < 30)]

#removing lines with missing data
weather_choice = weather_choice.dropna()

weather_choice

,Unnamed: 0,City,Clouds,Country,Date,Latitude,Longitude,High Temperature,Wind Speed,Humidity
237,237,hofn,6,IS,1618767910,64.2539,-15.2082,5.40,14.04,63
288,288,san bartolome de tirajana,20,ES,1618767973,27.9248,-15.5733,21.11,12.35,60
354,354,wajima,78,JP,1618768057,37.4000,136.9000,10.99,10.24,69
421,421,vila do maio,20,CV,1618768141,15.1333,-23.2167,25.00,10.80,61
535,535,bullhead city,1,US,1618768288,35.1478,-114.5683,23.89,11.83,12


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [57]:
url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

hotel_df = pd.DataFrame(columns = ['Hotel Name', 'Latitude', 'Longitude', 'Address', 'City', 'Country'])

for x, city in weather_choice.iterrows():
    
    specs = {"location": f'{city["Latitude"]},{city["Longitude"]}',
            "keyword": "hotel",
            "radius": 5000,
            "key": g_key}


    try:
        hotels = requests.get(url, params = specs)
        hotels_json = hotels.json()
        hotel_df = hotel_df.append({"Hotel Name":hotels_json["results"][0]["name"],
                         "Latitude":city['Latitude'],
                         "Longitude": city['Longitude'],
                         "Address": hotels_json["results"][0]["vicinity"],
                                  "Country": city["Country"], 
                                  "City": city["City"]},ignore_index=True)
        print(f"Hotel(s) found in {city['City']}")
    except:
        print(f"Bring a tent, no hotels within 5000 meters of {city['City']}")

Hotel(s) found in hofn
Hotel(s) found in san bartolome de tirajana
Hotel(s) found in wajima
Hotel(s) found in vila do maio
Hotel(s) found in bullhead city


In [58]:
hotel_df

,Hotel Name,Latitude,Longitude,Address,City,Country
0,Hótel Höfn,64.2539,-15.2082,"Víkurbraut 20, Höfn í Hornafirði",hofn,IS
1,Las Tirajanas,27.9248,-15.5733,"C/ Oficial Mayor Jose Rubio s/n, San Bartolomé...",san bartolome de tirajana,ES
2,Wajiima Onsen Yashio,37.4000,136.9000,"Sodegahama-1 Fugeshimachi, Wajima",wajima,JP
3,Big Game Maio,15.1333,-23.2167,Vila do Maio,vila do maio,CV
4,Don Laughlin's Riverside Resort Hotel and Casino,35.1478,-114.5683,"1650 S Casino Dr, Laughlin",bullhead city,US


In [60]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [62]:
# Add marker layer ontop of heat map
hotelmap = gmaps.figure()

hotel_markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
hmap.add_layer(hotel_markers)

# Display figure
hmap

Figure(layout=FigureLayout(height='420px'))